In [58]:
import operator
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition, create_react_agent
from langgraph.types import Command
from langgraph.graph import MessagesState, add_messages
from pydantic import BaseModel, Field
from typing import Annotated, Sequence, Optional, Dict, Any, TypedDict, Literal, List
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import PromptTemplate
import json
from langchain.tools import BaseTool
from Bio import Entrez
import xml.etree.ElementTree as ET
import yfinance as yf
import pandas as pd

### Initializing the LLMs

In [2]:
## Langchain and Langsmith tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ["LANGCHAIN_TRACING_V2"]="true"

## Getting Froq API key
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# EMBEDDING_MODEL = "BAAI/bge-small-en"
GROQ_INFERENCE_MODEL = "deepseek-r1-distill-llama-70b"
OPENAI_INFERENCE_MODEL = "gpt-4o"
# INFERENCE_MODEL = "gemma2-9b-it"

In [3]:
# llm = ChatGroq(model=GROQ_INFERENCE_MODEL, temperature=0.3)
# print('--------Creating the response from llm based on contexts--------')

llm = ChatOpenAI(model=OPENAI_INFERENCE_MODEL, temperature=0.3)
print('--------Creating the response from llm based on contexts--------')

--------Creating the response from llm based on contexts--------


### State of the Agents

In [4]:
class SupervisorState(MessagesState):
    """State of the Supervisor Agent"""
    query: str
    research_results: Optional[Dict[str, Any]] = None
    summary_results: Optional[Dict[str, Any]] = None
    final_documents: Optional[str] = None
    next: Optional[Literal['research', 'report', 'FINISH']] = None
    reasoning: Optional[str] = None
    task_details: Optional[str] = None


In [5]:
class ResearchState(MessagesState):
    """State of the Team 1: Research Agent"""
    query: str
    next: Optional[Literal['finance', 'pharma']] = None
    reasoning: Optional[str] = None
    task_details: Optional[str] = None
    research_context: str
    medical_results: Optional[Dict[str, Any]]
    financial_results: Optional[Dict[str, Any]]

In [6]:
class ReportingState(MessagesState):
    """State of the Team 2: Reporting Agent"""
    query: str
    next: Optional[Literal['summarization', 'doc_generation']] = None
    reasoning: str
    task_details: str
    research_findings: str = ""
    report_type: Optional[Literal['summary','document']] = None
    summary: Optional[str]
    document_content: str = ""
    has_summary: bool = False
    # report_format:Optional[Literal['pdf','docx','html','markdown']] = None
    # research_data: Dict[str, Any]
    report_context: str = ""
    report_requirements: str = ""
    final_report: Optional[str]

### Router of the Agents

In [7]:
class SupervisorRouter(TypedDict):
    next: Literal['research', 'report', 'FINISH']
    reasoning: str
    task_details: str

In [8]:
class ResearchRouter(TypedDict):
    next: Literal['pharma', 'financial']
    reasoning: str
    task_details: str
    research_context: str

In [9]:
class ReportRouter(TypedDict):
    next: Literal['summary', 'document']
    reasoning: str
    report_requirements: str
    report_context: str
    report_format: str

### Agent Prompt Template

#### supervisor prompt template

In [10]:
supervisor_prompt_template = """You are the Supervisor Agent coordinating a hierarchical multi-agent research system.

Your responsibilities:
1. Analyze incoming requests to determine if research or reporting is needed
2. Coordinate between Team 1 (Research) and Team 2 (Reporting)
3. Make decisions on workflow progression

Teams under your supervision:
- Team 1 (Research): Coordinates medical/pharma (Team 3) and financial (Team 4) research
- Team 2 (Reporting): Coordinates summary creation (Team 5) and document generation (Team 6)

Decision criteria:
- If no research results exist and task requires data gathering then route to "research" agent i.e. Team 1
- If research results exist but no report generated then route to "reporting" agent i.e. Team 2  
- If final document is complete then route to "end"

Always respond with your decision and reasoning in JSON format:
{{  "next_action": "research|reporting|end", 
    "reasoning": "explanation", 
    "task_details": "specific instructions"}}

Here is the user query: {question}
"""

In [11]:
supervisor_llm_with_structured_output = llm.with_structured_output(SupervisorRouter)
formatted_prompt = supervisor_prompt_template.format(question="What are the components of the medicine Ecosprin 75?")
response = supervisor_llm_with_structured_output.invoke(formatted_prompt)

In [12]:
response

{'next': 'research',
 'reasoning': 'The query requires gathering specific medical information about the components of Ecosprin 75, which is a task for the Research team. There is no indication that existing research results are available, necessitating a fresh research effort.',
 'task_details': 'Team 1 should coordinate with Team 3 (Medical/Pharma Research) to gather detailed information on the components of the medicine Ecosprin 75. Once the research is complete, the findings should be documented for further processing.'}

#### researcher prompt template

In [13]:
researcher_prompt_template = """You are a research coordinator that determines which specialized research agent should handle a query.

Your responsibilities:
1. Analyze the incoming query and determine whether it requires:
    - "finance": Financial research, market analysis, economic data, investment research, financial modeling, etc.
    - "pharma": Pharmaceutical research, drug development, clinical trials, regulatory affairs, medical research, etc.
2. Coordinate between Team 3 (Pharma Specialist Researcher) and Team 4 (Finance Specialist Researcher)
3. Consolidate results from specialized research
4. Report back to Supervisor

Teams under your supervision:
- Team 3 (Pharma Specialist Researcher): Perform Pharma Related Research and returns query related research information in the field of pharma and medicine
- Team 4 (Finance Specialist Researcher): Perform Finance Related Research and returns query related research information in the field of finance

Consider the following:
- What type of specialized knowledge is required?
- Which domain expertise would be most valuable?
- Are there any domain-specific terminologies or concepts?
- What kind of data sources would be most relevant?
    
Provide your routing decision with clear reasoning and specific task details for the chosen agent.

Query: {question}
Task Details: {task_details}
"""

In [14]:
researcher_llm_with_structured_output = llm.with_structured_output(ResearchRouter)
research_formatted_prompt = researcher_prompt_template.format(question="What are the components of the medicine Ecosprin 75?",
                                                     task_details="Investigate and gather information on the components of Ecosprin 75.")
response = researcher_llm_with_structured_output.invoke(research_formatted_prompt)
response

{'next': 'pharma',
 'reasoning': 'The query specifically asks about the components of a medicine, Ecosprin 75, which falls under the domain of pharmaceutical research. This requires specialized knowledge in drug composition and medical formulations.',
 'task_details': 'Team 3 (Pharma Specialist Researcher) is tasked with investigating and gathering detailed information on the components of the medicine Ecosprin 75. This includes identifying the active and inactive ingredients, understanding their roles, and any relevant pharmacological data.',
 'research_context': 'Ecosprin 75 is a medication used primarily for its antiplatelet properties, often prescribed to prevent heart attacks and strokes. Understanding its composition is crucial for healthcare professionals and patients to ensure its safe and effective use.'}

#### reporter prompt template

In [15]:
reporter_prompt_template = """You are a report coordinator that determines the appropriate reporting workflow based on the research findings and requirements.

Analyze the request and determine the reporting workflow:

DECISION RULES:
- If NO summary exists AND user wants a document: Route to "summary" first
- If summary exists AND user wants a document: Route to "document" 
- If user only wants a summary: Route to "summary"
- If unclear, default to "summary" first

Consider the following factors:
- What type of output does the user need? (summary, full document, both)
- What format is most appropriate? (markdown, PDF, HTML, etc.)
- How detailed should the report be?
- What is the intended audience?
- Are there specific requirements mentioned?

Provide your routing decision with clear reasoning and specific requirements for the chosen agent.

Query: {question}
Task Details: {task_details}
Research Findings: {research_findings}
"""

In [16]:
reporter_llm_with_structured_output = llm.with_structured_output(ReportRouter)
reporter_formatted_prompt = reporter_prompt_template.format(question="What are the components of the medicine Ecosprin 75?",
                                                     task_details="Investigate and gather information on the components of Ecosprin 75.",
                                                     research_findings="Ecosprin 75 helps in keeping blood thinner")
response = reporter_llm_with_structured_output.invoke(reporter_formatted_prompt)
response

{'next': 'summary',
 'reasoning': "Given the query, the user is seeking information about the components of Ecosprin 75. The request is straightforward and does not specify a need for a full document or detailed analysis. Additionally, the research findings provided are minimal, indicating that a summary would suffice to address the user's needs.",
 'report_requirements': '1. Identify and list the active and inactive ingredients in Ecosprin 75. \n2. Provide a brief description of the primary function of each component, focusing on its role in blood thinning.\n3. Ensure the summary is concise and easy to understand for a general audience.',
 'report_context': 'The user is interested in understanding the components of Ecosprin 75, likely for general knowledge or personal health considerations. The summary should be accessible to individuals without a medical background.',
 'report_format': 'markdown'}

### Agents

#### Supervisor Agent

In [17]:
def supervisor(state:SupervisorState)->Command[Literal['research', 'report', '__end__']]:
    """This is the supervisor agent. It takes the query and redirects it to either the research node or reporting node"""
    supervisor_llm_with_structured_output = llm.with_structured_output(SupervisorRouter)
    formatted_prompt = supervisor_prompt_template.format(question=state["query"])
    response = supervisor_llm_with_structured_output.invoke(formatted_prompt)
    
    goto = response["next"]

    if goto == "FINISH":
        goto=END
    
    supervisor_message = AIMessage(
        content=f"SUPERVISOR DECISION: Routing to {goto}. Reasoning: {response['reasoning']}. Task Details: {response['task_details']}"
    )
    
    return Command(goto=goto, update={
        "query": state["query"],
        "next": goto,
        "reasoning": response["reasoning"],
        "task_details": response["task_details"],
        "messages": [supervisor_message]})
    

#### Researcher Agent

In [36]:
def researcher(state:ResearchState)->Command[Literal['pharma','finance']]:
    """This is the supervisor agent. It takes the query from the supervisor agent, analyses whether it is related to pharma or finance and redirects it to either to pharma agent and finance agent."""
    query = state["query"]
    task_details = state["task_details"]
    researcher_llm_with_structured_output = llm.with_structured_output(ResearchRouter)
    formatted_prompt = researcher_prompt_template.format(question=query, task_details=task_details)
    response = researcher_llm_with_structured_output.invoke(formatted_prompt)

    goto = response["next"]

    if goto == "FINISH":
        goto=END
    
    researcher_message = AIMessage(
        content=f"RESEARCHER DECISION: Routing to {goto}. Reasoning: {response['reasoning']}. Research Context: {response['research_context']}. Task Details: {response['task_details']}"
    )

    return Command(goto=goto, update={
        "query": state["query"],
        "next": goto,
        "reasoning": response["reasoning"],
        "task_details": response["task_details"],
        "research_context": response["research_context"],
        "messages": add_messages([researcher_message])})
    

#### Reporter Agent

In [37]:
def reporter(state:ReportingState):
    """This is the supervisor agent. It takes the query from the supervisor agent, analyses whether it is related to pharma or finance and redirects it to either to pharma agent and finance agent."""
    query = state["query"]
    task_details = state["task_details"]
    research_findings = state["research_findings"]
    reporter_llm_with_structured_output = llm.with_structured_output(ReportRouter)
    reporter_formatted_prompt = reporter_prompt_template.format(question=query,
                                                     task_details=task_details,
                                                     research_findings=research_findings
                                                     )
    response = reporter_llm_with_structured_output.invoke(reporter_formatted_prompt)
    
    goto = response["next"]

    if goto == "FINISH":
        goto=END
    
    reporter_message = AIMessage(
        content=f"REPORTER DECISION: Routing to {goto}. Reasoning: {response['reasoning']}. Report Requirements: {response['report_requirements']}. Report Fomat: {response['report_format']}"
    )

    return Command(goto=goto, update={
        "query": state["query"],
        "next": goto,
        "reasoning": response["reasoning"],
        "task_details": response["task_details"],
        "research_context": response["research_context"],
        "research_findings": state.get("research_findings", ""),
        "report_type": goto,
        "report_context": response["report_context"],
        "report_requirements": response["report_requirements"],
        "report_format": response["report_format"],
        "messages": add_messages([reporter_message])})

#### Medical Researcher Agent

In [63]:
def make_system_prompt(instruction:str)->str:
    return  (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to gather as much information related to the question."
        " Execute what you can to make progress."
        f"\n{instruction}"
    )

In [ ]:
def medical_researcher(state:ResearchState)->Command[Literal["research"]]:
    med_web_tool = MedicalWebSearchTool()
    drug_tool = DrugTool()
    
    # llm.bind_tools([med_web_tool, drug_tool]).invoke("paracetamol")
    medical_research_agent=create_react_agent(
        llm,
        tools=[med_web_tool, drug_tool],
        prompt="""You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to gather as much information related to the question in the field of pharma and medical domain. Execute what you can to make progress and provide an answer"""
        )
    
    result=medical_research_agent.invoke(state["query"])
    # goto=get_next_node(result["messages"][-1],"chart_generator")
    # result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="researcher")
    # return Command(update={"messages": result["messages"]},goto=goto)
    return Command(
        update={
            "medical_results": [
                HumanMessage(content=result["messages"][-1].content, name="medical_researcher")
            ]
        },
        goto="research",
    )

In [61]:
med_web_tool = MedicalWebSearchTool()
# drug_tool = DrugTool()

# llm.bind_tools([med_web_tool, drug_tool]).invoke("paracetamol")
medical_research_agent=create_react_agent(
    llm,
    tools=[med_web_tool],
    prompt="""You are a helpful AI assistant, collaborating with other assistants. Use the provided tools to gather as much information related to the question in the field of pharma and medical domain. Execute what you can to make progress and provide an answer"""
    )

result=medical_research_agent.invoke({"query":"Aspirin"})

In [62]:
result

{'messages': [AIMessage(content='What specific question or topic would you like to explore in the pharma and medical domain?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 119, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BpcuTYrZ6rfOHGuLLLTgsvsFtMRfq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6a28133e-1d09-4b6b-9c86-55e420008a85-0', usage_metadata={'input_tokens': 119, 'output_tokens': 18, 'total_tokens': 137, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

#### Finance Researcher Agent

In [65]:
def finance_researcher(state:ResearchState)->Command[Literal["research"]]:
    stock_data_tool = StockDataTool()
    company_info_tool = CompanyInfoTool()
    fin_web_search = FinancialWebSearchTool(serpapi_api_key=os.getenv("SERP_API_KEY"))
    # llm.bind_tools([med_web_tool, drug_tool]).invoke("paracetamol")
    finance_research_agent=create_react_agent(
        llm,
        tools=[stock_data_tool, company_info_tool, fin_web_search],
        prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ), 
        )
    
    result=finance_research_agent.invoke(state["query"])
    # goto=get_next_node(result["messages"][-1],"chart_generator")
    # result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="researcher")
    # return Command(update={"messages": result["messages"]},goto=goto)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="finance_researcher")
            ]
        },
        goto="research",
    )

#### Document Summarizer Agent

In [11]:
def document_summarizer(state:AgentState):
    pass

#### Document Generator Agent

In [12]:
def document_generator(state:AgentState):
    pass

### Workflow

In [14]:
graph_builder = StateGraph(AgentState)
graph_builder.add_node("Researcher", researcher)
graph_builder.add_node("Reporter", reporter)
graph_builder.add_node("Supervisor",supervisor)
graph_builder.add_node("Medical Researcher", medical_researcher)
graph_builder.add_node("Finance Researcher", finance_researcher)
graph_builder.add_node("Document Summarizer", document_summarizer)
graph_builder.add_node("Document Generator", document_generator)
graph_builder.add_edge(START, "Supervisor")
# graph_builder.add_conditional_edges("ai_assistant",
#                                     tools_condition)
# graph_builder.add_edge("tools","ai_assistant")
# app_react = graph_builder.compile(checkpointer=memory, interrupt_before=["tools"])

## Tools

### Medical Tool

##### PubMed LangChain Tool

In [57]:
# !pip install xmltodict

In [62]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun
pubmed_query_tool = PubmedQueryRun()
pprint(pubmed_query_tool.invoke("Aspirin"))

('Published: 2025-07-04\n'
 'Title: Institutional Experience in Aspirin-Exacerbated Respiratory Disease '
 'Yields Favorable Pulmonary and Sinonasal Outcomes.\n'
 'Copyright Information: \n'
 'Summary::\n'
 'BACKGROUND: Aspirin-exacerbated respiratory disease (AERD) is characterized '
 'by asthma, chronic rhinosinusitis with polyps, and sensitivity to aspirin '
 '(ASA). Optimal treatment requires coordinated medical and surgical '
 'management, with prior studies showing that a single-center approach to AERD '
 'management yields improved sinonasal patient outcomes. Here, we sought to '
 'evaluate whether institutional experience enhances pulmonary and sinonasal '
 'outcomes in patients with AERD undergoing functional endoscopic sinus '
 'surgery (FESS) followed by ASA desensitization (AD).\n'
 'METHODS: Single-center, retrospective cohort study evaluating patients '
 'undergoing FESS and AD from 2016 to 2024. Cohorts were defined as "early" '
 '(2016-2019) and "late" (2020-2024). Demo

In [ ]:
# Queries like a molecular composition of a drug or clinical research related query cannot be answered, hence a more enhanced tool is required

#### Medical Web Search Tool(For Future Implementation)

##### Test

In [17]:
# !pip install Bio

In [18]:
from Bio import Entrez, Medline
Entrez.email = "researcher@example.com"
handle = Entrez.esearch(db="pmc", term="ecosprin", retmax=3, sort="relevance")
search_results = Entrez.read(handle)
handle.close()

search_results

{'Count': '169', 'RetMax': '3', 'RetStart': '0', 'IdList': ['10259437', '10824206', '11128359'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'ecosprin[All Fields]', 'Field': 'All Fields', 'Count': '170', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'ecosprin[All Fields]'}

In [19]:
if search_results.get("IdList"):
    handle = Entrez.efetch(
        db="pmc",
        id=search_results["IdList"],
        retmode="xml")

In [20]:
import xml.etree.ElementTree as ET
xml_data = handle.read()
handle.close()
root = ET.fromstring(xml_data)
results = []
for article in root.findall(".//article"):
    # Extract article metadata
    title_elem = article.find(".//article-title")
    abstract_elem = article.find(".//abstract")
    journal_elem = article.find(".//journal-title")
    pmcid_elem = article.find(".//article-id[@pub-id-type='pmc']")
    pmid_elem = article.find(".//article-id[@pub-id-type='pmid']")
    
    # Extract authors
    authors = []
    for author in article.findall(".//contrib[@contrib-type='author']"):
        given_name = author.find(".//given-names")
        surname = author.find(".//surname")
        if given_name is not None and surname is not None:
            authors.append(f"{given_name.text} {surname.text}")
    
    # Extract publication date
    pub_date = ""
    date_elem = article.find(".//pub-date")
    if date_elem is not None:
        year = date_elem.find(".//year")
        month = date_elem.find(".//month")
        day = date_elem.find(".//day")
        if year is not None:
            pub_date = year.text
            if month is not None:
                pub_date += f"-{month.text}"
            if day is not None:
                pub_date += f"-{day.text}"
    
    results.append({
        "source": "PubMed Central (PMC)",
        "title": title_elem.text if title_elem is not None else "",
        "authors": authors,
        "journal": journal_elem.text if journal_elem is not None else "",
        "publication_date": pub_date,
        "abstract": abstract_elem.text if abstract_elem is not None else "",
        "pmcid": pmcid_elem.text if pmcid_elem is not None else "",
        "pmid": pmid_elem.text if pmid_elem is not None else "",
        "doi": [],
        "keywords": []
    })

results

[{'source': 'PubMed Central (PMC)',
  'title': 'Triumph Over Adversity: A Comprehensive Case Series on Successful Pregnancy Outcomes in Antiphospholipid Antibody (APLA)-Positive Patients',
  'authors': ['Manju Mathesan', 'Shanthi Ethirajan'],
  'journal': 'Cureus',
  'publication_date': '2024-4-26',
  'abstract': None,
  'pmcid': '',
  'pmid': '38803763',
  'doi': [],
  'keywords': []},
 {'source': 'PubMed Central (PMC)',
  'title': 'Bullous hemorrhagic dermatosis: A rare cutaneous reaction of heparin',
  'authors': ['N Dhattarwal', 'R Gurjar'],
  'journal': 'Journal of Postgraduate Medicine',
  'publication_date': '2023',
  'abstract': None,
  'pmcid': '',
  'pmid': '36861545',
  'doi': [],
  'keywords': []},
 {'source': 'PubMed Central (PMC)',
  'title': 'Cardiac tamponade and basilar artery aneurysm following leptospirosis: A case report',
  'authors': ['Saumitra Misra',
   'Syed Nabeel Muzaffar',
   'Shubhajeet Roy',
   'Shashank Prajapati'],
  'journal': 'International Journal of 

In [21]:
import requests
search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
search_params = {
    "db": "pmc",
    "term": "ecosprin",
    "retmax": 3,
    "retmode": "json",
    "sort": "relevance"
}
search_response = requests.get(search_url, params=search_params)
if search_response.status_code == 200:
    search_data = search_response.json()

search_data

{'header': {'type': 'esearch', 'version': '0.3'},
 'esearchresult': {'count': '169',
  'retmax': '3',
  'retstart': '0',
  'idlist': ['10259437', '10824206', '11128359'],
  'translationset': [],
  'translationstack': [{'term': 'ecosprin[All Fields]',
    'field': 'All Fields',
    'count': '170',
    'explode': 'N'},
   'GROUP'],
  'querytranslation': 'ecosprin[All Fields]'}}

In [22]:
import requests
search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
search_params = {
    "db": "pmc",
    "term": "ecosprin",
    "retmax": 3,
    "retmode": "json",
    "sort": "relevance"
}
search_response = requests.get(search_url, params=search_params)
if search_response.status_code == 200:
    search_data = search_response.json()
    pmids = search_data.get("esearchresult", {}).get("idlist", [])
    
    if pmids:
        # Fetch detailed information
        fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        fetch_params = {
            "db": "pmc",
            "id": ",".join(pmids),
            "retmode": "xml"
        }
        
        fetch_response = requests.get(fetch_url, params=fetch_params)
        if fetch_response.status_code == 200:
            # Parse XML response with proper author extraction
            import xml.etree.ElementTree as ET
            root = ET.fromstring(fetch_response.content)
            
            results = []
            for article in root.findall(".//PubmedArticle"):
                # Extract basic info
                title_elem = article.find(".//ArticleTitle")
                abstract_elem = article.find(".//AbstractText")
                journal_elem = article.find(".//Journal/Title")
                pmid_elem = article.find(".//PMID")
                
                # Extract authors properly
                authors = []
                author_list = article.find(".//AuthorList")
                if author_list is not None:
                    for author in author_list.findall(".//Author"):
                        # Try to get last name and first name
                        last_name = author.find(".//LastName")
                        first_name = author.find(".//ForeName")
                        initials = author.find(".//Initials")
                        
                        if last_name is not None:
                            author_name = last_name.text
                            if first_name is not None:
                                author_name = f"{first_name.text} {author_name}"
                            elif initials is not None:
                                author_name = f"{initials.text} {author_name}"
                            authors.append(author_name)
                
                # Extract publication date
                pub_date = ""
                pub_date_elem = article.find(".//PubDate")
                if pub_date_elem is not None:
                    year = pub_date_elem.find(".//Year")
                    month = pub_date_elem.find(".//Month")
                    day = pub_date_elem.find(".//Day")
                    
                    if year is not None:
                        pub_date = year.text
                        if month is not None:
                            pub_date += f"-{month.text}"
                            if day is not None:
                                pub_date += f"-{day.text}"
                
                # Extract keywords/MeSH terms
                keywords = []
                mesh_list = article.find(".//MeshHeadingList")
                if mesh_list is not None:
                    for mesh in mesh_list.findall(".//MeshHeading"):
                        descriptor = mesh.find(".//DescriptorName")
                        if descriptor is not None:
                            keywords.append(descriptor.text)
                
                results.append({
                    "source": "PubMed (REST API)",
                    "title": title_elem.text if title_elem is not None else "",
                    "abstract": abstract_elem.text if abstract_elem is not None else "",
                    "journal": journal_elem.text if journal_elem is not None else "",
                    "pmid": pmid_elem.text if pmid_elem is not None else "",
                    "authors": authors,
                    "publication_date": pub_date,
                    "keywords": keywords
                })
        
results

[]

In [23]:
{'Count': '169', 'RetMax': '3', 'RetStart': '0', 'IdList': ['10259437', '10824206', '11128359'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'ecosprin[All Fields]', 'Field': 'All Fields', 'Count': '170', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'ecosprin[All Fields]'}

{'Count': '169',
 'RetMax': '3',
 'RetStart': '0',
 'IdList': ['10259437', '10824206', '11128359'],
 'TranslationSet': [],
 'TranslationStack': [{'Term': 'ecosprin[All Fields]',
   'Field': 'All Fields',
   'Count': '170',
   'Explode': 'N'},
  'GROUP'],
 'QueryTranslation': 'ecosprin[All Fields]'}

In [24]:
[{'source': 'PubMed', 'title': '[Effects of antisense human telomerase RNA on growth of human gastric cancer cells].', 'abstract': 'To observe the inhibitory effects of antisense human telomerase RNA (hTR) on growth of human gastric cancer cells.', 'journal': 'Zhonghua bing li xue za zhi = Chinese journal of pathology', 'pmid': '11866938', 'authors': [], 'publication_date': '', 'doi': [], 'keywords': []}, {'source': 'PubMed', 'title': 'Aminoglycoside dosing: time to change.', 'abstract': '', 'journal': 'Australian and New Zealand journal of medicine', 'pmid': '7980229', 'authors': [], 'publication_date': '', 'doi': [], 'keywords': []}]

[{'source': 'PubMed',
  'title': '[Effects of antisense human telomerase RNA on growth of human gastric cancer cells].',
  'abstract': 'To observe the inhibitory effects of antisense human telomerase RNA (hTR) on growth of human gastric cancer cells.',
  'journal': 'Zhonghua bing li xue za zhi = Chinese journal of pathology',
  'pmid': '11866938',
  'authors': [],
  'publication_date': '',
  'doi': [],
  'keywords': []},
 {'source': 'PubMed',
  'title': 'Aminoglycoside dosing: time to change.',
  'abstract': '',
  'journal': 'Australian and New Zealand journal of medicine',
  'pmid': '7980229',
  'authors': [],
  'publication_date': '',
  'doi': [],
  'keywords': []}]

In [25]:
url = "https://api.fda.gov/drug/label.json"
params = {
    "search": "paracetamol",
    "limit": 10
}

response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    results = []
    
    for result in data.get("results", []):
        results.append({
            "source": "FDA",
            "brand_name": result.get("openfda", {}).get("brand_name", []),
            "generic_name": result.get("openfda", {}).get("generic_name", []),
            "manufacturer": result.get("openfda", {}).get("manufacturer_name", []),
            "indication": result.get("indications_and_usage", []),
            "dosage": result.get("dosage_and_administration", []),
            "warnings": result.get("warnings", []),
            "adverse_reactions": result.get("adverse_reactions", [])
        })
    
results

[{'source': 'FDA',
  'brand_name': ['Pilocarpine Hydrochloride'],
  'generic_name': ['PILOCARPINE HYDROCHLORIDE'],
  'manufacturer': ['Bryant Ranch Prepack'],
  'indication': ["INDICATIONS AND USAGE: Pilocarpine hydrochloride tablets, USP are indicated for 1) the treatment of symptoms of dry mouth from salivary gland hypofunction caused by radiotherapy for cancer of the head and neck; and 2) the treatment of symptoms of dry mouth in patients with Sjogren's syndrome."],
  'dosage': ["DOSAGE AND ADMINISTRATION: Regardless of the indication, the starting dose in patients with moderate hepatic impairment should be 5 mg twice daily, followed by adjustment based on therapeutic response and tolerability. Patients with mild hepatic insufficiency do not require dosage reductions. The use of pilocarpine in patients with severe hepatic insufficiency is not recommended. If needed, refer to the Hepatic Insufficiency subsection of the Precautions section of this label for definitions of mild, modera

##### Actual

In [26]:
class MedicalWebSearchTool(BaseTool):
    """Specialized web search tool for medical information"""
    
    name: str = "medical_web_search"
    description: str = """
    Search medical databases and websites for current medical information.
    Covers PubMed, medical journals, FDA databases, and clinical trial registries.
    """
    def __init__(self):
        super().__init__()
    
    def _run(self, query: str, search_type: str = "pubmed") -> str:
        """Execute medical web search"""
        try:
            results = []

            if search_type == "pubmed":
                results.extend(self._search_pubmed(query))
            elif search_type == "fda":
                results.extend(self._search_fda(query))
            # else:
            #     # General medical search across all sources
            #     results.extend(self._search_pubmed(query))
            #     results.extend(self._search_fda(query))
            return json.dumps(results, indent=2)
        except Exception as e:
            print(f"Medical web search error: {e}")
            raise f"Error in medical search: {str(e)}"
    
    def _search_pubmed(self, query: str, max_results: int = 20) -> List[Dict]:
        """Search PubMed database"""
        try:
            Entrez.email = "researcher@example.com"  # Set your email

            # Search PubMed
            handle = Entrez.esearch(db="pmc", term=query, retmax=max_results, sort="relevance")
            search_results = Entrez.read(handle)
            handle.close()

            # Get detailed information from PMC
            if search_results.get("IdList"):
                handle = Entrez.efetch(db="pmc", id=search_results["IdList"], retmode="xml")
                
                xml_data = handle.read()
                handle.close()
                
                root = ET.fromstring(xml_data)
                results = []
                
                for article in root.findall(".//article"):
                    # Extract article metadata
                    title_elem = article.find(".//article-title")
                    abstract_elem = article.find(".//abstract")
                    journal_elem = article.find(".//journal-title")
                    pmcid_elem = article.find(".//article-id[@pub-id-type='pmc']")
                    pmid_elem = article.find(".//article-id[@pub-id-type='pmid']")
                    
                    # Extract authors
                    authors = []
                    for author in article.findall(".//contrib[@contrib-type='author']"):
                        given_name = author.find(".//given-names")
                        surname = author.find(".//surname")
                        if given_name is not None and surname is not None:
                            authors.append(f"{given_name.text} {surname.text}")
                    
                    # Extract publication date
                    pub_date = ""
                    date_elem = article.find(".//pub-date")
                    if date_elem is not None:
                        year = date_elem.find(".//year")
                        month = date_elem.find(".//month")
                        day = date_elem.find(".//day")
                        if year is not None:
                            pub_date = year.text
                            if month is not None:
                                pub_date += f"-{month.text}"
                            if day is not None:
                                pub_date += f"-{day.text}"
                    
                    results.append({
                        "source": "PubMed Central (PMC)",
                        "title": title_elem.text if title_elem is not None else "",
                        "authors": authors,
                        "journal": journal_elem.text if journal_elem is not None else "",
                        "publication_date": pub_date,
                        "abstract": abstract_elem.text if abstract_elem is not None else "",
                        "pmcid": pmcid_elem.text if pmcid_elem is not None else "",
                        "pmid": pmid_elem.text if pmid_elem is not None else "",
                        "doi": [],
                        "keywords": []
                    })
                return results
            else:
                return []
        except Exception as e:
            print(f"PubMed search error: {e}")
            raise f"Error in PubMed search: {str(e)}"
    
    def _search_fda(self, query: str) -> List[Dict]:
        """Search FDA databases"""
        try:
            # FDA Drug Database API
            url = "https://api.fda.gov/drug/label.json"
            params = {
                "search": f'"{query}"',
                "limit": 10
            }
            
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                print(f'data: {data}')
                results = []
                
                for result in data.get("results", []):
                    results.append({
                        "source": "FDA",
                        "brand_name": result.get("openfda", {}).get("brand_name", []),
                        "generic_name": result.get("openfda", {}).get("generic_name", []),
                        "manufacturer": result.get("openfda", {}).get("manufacturer_name", []),
                        "indication": result.get("indications_and_usage", []),
                        "dosage": result.get("dosage_and_administration", []),
                        "warnings": result.get("warnings", []),
                        "adverse_reactions": result.get("adverse_reactions", [])
                    })
                
                return results
                
        except Exception as e:
            print(f"FDA search error: {e}")
            raise f"Error in FDA search: {str(e)}"


In [27]:
tool = MedicalWebSearchTool()
result = tool.invoke({"query": "Give me the composition of Aspirin"})
result

'[\n  {\n    "source": "PubMed Central (PMC)",\n    "title": "Beyond COX-1: the effects of aspirin on platelet biology and potential mechanisms of chemoprevention",\n    "authors": [\n      "Argentina Ornelas",\n      "Niki Zacharias-Millward",\n      "David G. Menter",\n      "Jennifer S. Davis",\n      "Lenard Lichtenberger",\n      "David Hawke",\n      "Ernest Hawk",\n      "Eduardo Vilar",\n      "Pratip Bhattacharya",\n      "Steven Millward"\n    ],\n    "journal": "Cancer Metastasis Reviews",\n    "publication_date": "2017-7-31",\n    "abstract": null,\n    "pmcid": "",\n    "pmid": "28762014",\n    "doi": [],\n    "keywords": []\n  },\n  {\n    "source": "PubMed Central (PMC)",\n    "title": "Effect of dietary aspirin eugenol ester on the growth performance, antioxidant capacity, intestinal inflammation, and cecal microbiota of broilers under high stocking density",\n    "authors": [\n      "Haojie Zhang",\n      "Yi Zhang",\n      "Dongying Bai",\n      "Jiale Zhong",\n      

#### Drug Tool(For Future Implementation)

##### Test

In [28]:
# !pip install rdkit

In [29]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

In [30]:
def calculate_lipinski_violations(mol) -> int:
    """Calculate Lipinski Rule of Five violations"""
    violations = 0
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    
    if mw > 500: violations += 1
    if logp > 5: violations += 1
    if hbd > 5: violations += 1
    if hba > 10: violations += 1
    
    return violations

In [31]:
def calculate_veber_violations(mol) -> int:
    """Calculate Veber rule violations"""
    violations = 0
    rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    tpsa = Descriptors.TPSA(mol)
    
    if rotatable_bonds > 10: violations += 1
    if tpsa > 140: violations += 1
    
    return violations

In [32]:
mol = Chem.MolFromSmiles("CCO")

properties = {
    "molecular_weight": Descriptors.MolWt(mol),
    "logp": Descriptors.MolLogP(mol),
    "hbd": Descriptors.NumHDonors(mol),
    "hba": Descriptors.NumHAcceptors(mol),
    "tpsa": Descriptors.TPSA(mol),
    "rotatable_bonds": Descriptors.NumRotatableBonds(mol),
    "aromatic_rings": Descriptors.NumAromaticRings(mol),
    "lipinski_violations": calculate_lipinski_violations(mol),
    "veber_violations": calculate_veber_violations(mol)
}

properties

{'molecular_weight': 46.069,
 'logp': -0.0014000000000000123,
 'hbd': 1,
 'hba': 1,
 'tpsa': 20.23,
 'rotatable_bonds': 0,
 'aromatic_rings': 0,
 'lipinski_violations': 0,
 'veber_violations': 0}

In [33]:
# !pip install pubchempy

In [34]:
import pubchempy as pcp

In [35]:
compounds = pcp.get_compounds("paracetamol", 'name', listkey_count=10)
results = []

for compound in compounds:
    results.append({
        "cid": compound.cid,
        "iupac_name": compound.iupac_name,
        "molecular_formula": compound.molecular_formula,
        "molecular_weight": compound.molecular_weight,
        "smiles": compound.canonical_smiles,
        "inchi": compound.inchi,
        "synonyms": compound.synonyms[:5] if compound.synonyms else []
    })

results

[{'cid': 1983,
  'iupac_name': 'N-(4-hydroxyphenyl)acetamide',
  'molecular_formula': 'C8H9NO2',
  'molecular_weight': '151.16',
  'smiles': None,
  'inchi': 'InChI=1S/C8H9NO2/c1-6(10)9-7-2-4-8(11)5-3-7/h2-5,11H,1H3,(H,9,10)',
  'synonyms': ['acetaminophen',
   'Paracetamol',
   '103-90-2',
   'Doliprane',
   'Injectapap']}]

##### Actual

In [36]:
class DrugTool(BaseTool):
    """Tool for drug discovery and molecular analysis"""
    
    name: str = "drug_discovery"
    description: str = """
    Analyze molecular structures, predict drug properties, and search chemical databases.
    Includes ADMET prediction, similarity search, and pharmacophore analysis.
    """

    def _run(self, query: str, analysis_type: str = "properties") -> str:
        """Execute drug discovery analysis"""
        try:
            if analysis_type == "properties":
                return self._analyze_molecular_properties(query)
            ## Future development
            # elif analysis_type == "similarity":
            #     return self._similarity_search(query)
            # elif analysis_type == "admet":
            #     return self._predict_admet(query)
            elif analysis_type == "pubchem":
                return self._search_pubchem(query)
                
        except Exception as e:
            print(f"Drug discovery error: {e}")
            raise f"Error in drug discovery analysis: {str(e)}"
    
    def _analyze_molecular_properties(self, smiles: str) -> str:
        """Analyze molecular properties from SMILES(Simplified Molecular Input Line Entry System)"""
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return "Invalid SMILES string"
            
            properties = {
                "molecular_weight": Descriptors.MolWt(mol),
                "logp": Descriptors.MolLogP(mol),
                "hbd": Descriptors.NumHDonors(mol),
                "hba": Descriptors.NumHAcceptors(mol),
                "tpsa": Descriptors.TPSA(mol),
                "rotatable_bonds": Descriptors.NumRotatableBonds(mol),
                "aromatic_rings": Descriptors.NumAromaticRings(mol),
                "lipinski_violations": self._calculate_lipinski_violations(mol),
                "veber_violations": self._calculate_veber_violations(mol)
            }
            
            return json.dumps(properties, indent=2)
            
        except Exception as e:
            print(f"Molecular properties error: {e}")
            raise f"Error cin fetching molecular properties: {str(e)}"
    
    def _calculate_lipinski_violations(self, mol) -> int:
        """Calculate Lipinski Rule of Five violations"""
        violations = 0
        mw = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        hbd = Descriptors.NumHDonors(mol)
        hba = Descriptors.NumHAcceptors(mol)
        
        if mw > 500: violations += 1
        if logp > 5: violations += 1
        if hbd > 5: violations += 1
        if hba > 10: violations += 1
        
        return violations
    
    
    def _calculate_veber_violations(self, mol) -> int:
        """Calculate Veber rule violations"""
        violations = 0
        rotatable_bonds = Descriptors.NumRotatableBonds(mol)
        tpsa = Descriptors.TPSA(mol)
        
        if rotatable_bonds > 10: violations += 1
        if tpsa > 140: violations += 1
        
        return violations
    
    def _search_pubchem(self, query: str) -> str:
        """Search PubChem database"""
        try:
            compounds = pcp.get_compounds(query, 'name', listkey_count=10)
            results = []
            
            for compound in compounds:
                results.append({
                    "cid": compound.cid,
                    "iupac_name": compound.iupac_name,
                    "molecular_formula": compound.molecular_formula,
                    "molecular_weight": compound.molecular_weight,
                    "smiles": compound.canonical_smiles,
                    "inchi": compound.inchi,
                    "synonyms": compound.synonyms[:5] if compound.synonyms else []
                })
            
            return json.dumps(results, indent=2)
            
        except Exception as e:
            print(f"PubChem search error: {e}")
            raise f"Error searching PubChem: {str(e)}"

In [37]:
drug_tool = DrugTool()
result = drug_tool.invoke({"query":"CCO", "analysis_type":"pubchem"})
result

'[\n  {\n    "cid": 446195,\n    "iupac_name": "2-[[3-(3-chlorophenyl)-1,2,4-oxadiazol-5-yl]sulfanyl]acetic acid",\n    "molecular_formula": "C10H7ClN2O3S",\n    "molecular_weight": "270.69",\n    "smiles": null,\n    "inchi": "InChI=1S/C10H7ClN2O3S/c11-7-3-1-2-6(4-7)9-12-10(16-13-9)17-5-8(14)15/h1-4H,5H2,(H,14,15)",\n    "synonyms": [\n      "338746-11-5",\n      "2-{[3-(3-chlorophenyl)-1,2,4-oxadiazol-5-yl]sulfanyl}acetic acid",\n      "CARBOXYMETHYLTHIO-3-(3-CHLOROPHENYL)-1,2,4-OXADIAZOL",\n      "2-[[3-(3-chlorophenyl)-1,2,4-oxadiazol-5-yl]sulfanyl]acetic acid",\n      "2-((3-(3-chlorophenyl)-1,2,4-oxadiazol-5-yl)sulfanyl)acetic acid"\n    ]\n  }\n]'

In [38]:
result = drug_tool.invoke({"query":"CCO"})
result

'{\n  "molecular_weight": 46.069,\n  "logp": -0.0014000000000000123,\n  "hbd": 1,\n  "hba": 1,\n  "tpsa": 20.23,\n  "rotatable_bonds": 0,\n  "aromatic_rings": 0,\n  "lipinski_violations": 0,\n  "veber_violations": 0\n}'

In [39]:
med_web_tool = MedicalWebSearchTool()
drug_tool = DrugTool()
llm.bind_tools([med_web_tool, drug_tool]).invoke("paracetamol")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bKGmQQD12MOxCYDEBjNmyckQ', 'function': {'arguments': '{"query": "paracetamol", "search_type": "pubmed"}', 'name': 'medical_web_search'}, 'type': 'function'}, {'id': 'call_S7lcf1GyS4WeeRIqL8KNjC88', 'function': {'arguments': '{"query": "paracetamol", "analysis_type": "properties"}', 'name': 'drug_discovery'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 134, 'total_tokens': 197, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'id': 'chatcmpl-BpcvQI3Ii9ee0aaJmZemXkeYrx4iV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c71213ad-ae2b-43a4-b491-a9545773bc37-0', tool_calls=[{'name': 'm

#### Bio Informatics Tool(For Future Implementation)

##### Test

##### Actual

In [40]:
class BioinformaticsTool(BaseTool):
    """Tool for bioinformatics analysis"""
    
    name: str = "bioinformatics"
    description: str = """
    Perform bioinformatics analysis including sequence analysis, protein structure
    prediction, pathway analysis, and genomic data interpretation.
    """
    
    def _run(self, query: str, analysis_type: str = "sequence") -> str:
        """Execute bioinformatics analysis"""
        try:
            if analysis_type == "sequence":
                return self._sequence_analysis(query)
            # Future enhancements
            elif analysis_type == "protein":
                return self._protein_analysis(query)
            elif analysis_type == "pathway":
                return self._pathway_analysis(query)
            elif analysis_type == "genomics":
                return self._genomics_analysis(query)
                
        except Exception as e:
            print(f"Bioinformatics error: {e}")
            raise f"Error in bioinformatics analysis: {str(e)}"

### Finance Tool

In [65]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
yahoo_finance_tool = YahooFinanceNewsTool()
yahoo_finance_tool.invoke("TCS")

"Bear of the Day: The Container Store Group, Inc. (TCS)\nThe Container Store Group's recent drop is part of a massive long-term decline that was only interrupted a few times.\n\nTCS Q2 Earnings Miss Estimates, New Partnership With Beyond\nThe Container Store's fiscal Q2 results reflect continued sequential improvement, paving the way for long-term opportunities from its new partnership with Beyond, Inc."

In [73]:
# !pip install google-search-results

In [72]:
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper

google_finance_tool = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper(serp_api_key="cdf009c33c4cb3f472c7cef9b98212c90abb9f76f8ede5ec5c8c5ebb2c79395e"))
google_finance_tool.invoke("Give me the company info of SBIN")

'\nQuery: Give me the company info of SBIN\nNo summary information\neurope: price = 23787.45, movement = Down\nasia: price = 39810.88, movement = Up\n'

In [74]:
# Queries like a company information, or financial news requires some more specialized tool. Enhancement for future

#### Stock Data Tool

In [41]:
# !pip install yfinance

In [42]:
class StockDataTool(BaseTool):
    name: str = "stock_data_tool"
    description: str = "Fetch comprehensive stock data including price history, volume, and technical indicators"
    # args_schema = StockDataInput
    def _run(self, symbol: str, period: str = "1y", interval: str = "1d") -> str:
        try:
            ticker = yf.Ticker(symbol)
            hist = ticker.history(period=period, interval=interval)
            
            if hist.empty:
                return f"No data found for symbol: {symbol}"
            
            # Calculate technical indicators
            hist['SMA_20'] = hist['Close'].rolling(window=20).mean()
            hist['SMA_50'] = hist['Close'].rolling(window=50).mean()
            
            # Get current price and stats
            current_price = hist['Close'].iloc[-1]
            price_change = hist['Close'].iloc[-1] - hist['Close'].iloc[-2]
            price_change_pct = (price_change / hist['Close'].iloc[-2]) * 100
            
            # Volume analysis
            avg_volume = hist['Volume'].mean()
            current_volume = hist['Volume'].iloc[-1]
            
            result = {
                "symbol": symbol,
                "current_price": round(current_price, 2),
                "price_change": round(price_change, 2),
                "price_change_percent": round(price_change_pct, 2),
                "volume": int(current_volume),
                "average_volume": int(avg_volume),
                "52_week_high": round(hist['High'].max(), 2),
                "52_week_low": round(hist['Low'].min(), 2),
                "sma_20": round(hist['SMA_20'].iloc[-1], 2) if not pd.isna(hist['SMA_20'].iloc[-1]) else None,
                "sma_50": round(hist['SMA_50'].iloc[-1], 2) if not pd.isna(hist['SMA_50'].iloc[-1]) else None,
                "data_points": len(hist)
            }
            
            return json.dumps(result, indent=2)
            
        except Exception as e:
            return f"Error fetching stock data for {symbol}: {str(e)}"

In [43]:
stock_tool = StockDataTool()
stock_tool.invoke({"symbol":"SBIN.NS"})

'{\n  "symbol": "SBIN.NS",\n  "current_price": 811.85,\n  "price_change": 4.75,\n  "price_change_percent": 0.59,\n  "volume": 4317408,\n  "average_volume": 12861093,\n  "52_week_high": 881.3,\n  "52_week_low": 666.61,\n  "sma_20": 803.45,\n  "sma_50": 794.87,\n  "data_points": 250\n}'

#### Company Info Tool

In [44]:
class CompanyInfoTool(BaseTool):
    name: str = "company_info_tool"
    description: str = "Get detailed company information including business summary, financial metrics, and key statistics"
    # args_schema = CompanyInfoInput
    
    def _run(self, symbol: str) -> str:
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            
            # Extract key information
            company_info = {
                "symbol": symbol,
                "company_name": info.get("longName", "N/A"),
                "sector": info.get("sector", "N/A"),
                "industry": info.get("industry", "N/A"),
                "business_summary": info.get("longBusinessSummary", "N/A")[:500] + "..." if len(info.get("longBusinessSummary", "")) > 500 else info.get("longBusinessSummary", "N/A"),
                "market_cap": info.get("marketCap", "N/A"),
                "enterprise_value": info.get("enterpriseValue", "N/A"),
                "pe_ratio": info.get("trailingPE", "N/A"),
                "forward_pe": info.get("forwardPE", "N/A"),
                "price_to_book": info.get("priceToBook", "N/A"),
                "debt_to_equity": info.get("debtToEquity", "N/A"),
                "return_on_equity": info.get("returnOnEquity", "N/A"),
                "revenue": info.get("totalRevenue", "N/A"),
                "profit_margin": info.get("profitMargins", "N/A"),
                "dividend_yield": info.get("dividendYield", "N/A"),
                "beta": info.get("beta", "N/A"),
                "52_week_high": info.get("fiftyTwoWeekHigh", "N/A"),
                "52_week_low": info.get("fiftyTwoWeekLow", "N/A"),
                "employees": info.get("fullTimeEmployees", "N/A"),
                "website": info.get("website", "N/A")
            }
            
            return json.dumps(company_info, indent=2)
            
        except Exception as e:
            return f"Error fetching company info for {symbol}: {str(e)}"

In [45]:
from pprint import pprint
company_info_tool = CompanyInfoTool()
result = company_info_tool.invoke({"symbol":"SBIN.NS"})
pprint(result)

('{\n'
 '  "symbol": "SBIN.NS",\n'
 '  "company_name": "State Bank of India",\n'
 '  "sector": "Financial Services",\n'
 '  "industry": "Banks - Regional",\n'
 '  "business_summary": "State Bank of India provides banking products and '
 'services in India and internationally. It operates through the Treasury, '
 'Corporate/Wholesale Banking, Retail Banking, and Other Banking Business '
 'segments. The company offers personal banking products and services, '
 'including current, savings, salary, and deposit accounts; home, personal, '
 'pension, auto, education, and gold loans, as well as loans against insurance '
 'policies, property, and securities; debit, business debit, prepaid, and '
 'green remit c...",\n'
 '  "market_cap": 7245452541952,\n'
 '  "enterprise_value": 12142308753408,\n'
 '  "pe_ratio": 9.340198,\n'
 '  "forward_pe": 9.82037,\n'
 '  "price_to_book": 1.4877903,\n'
 '  "debt_to_equity": "N/A",\n'
 '  "return_on_equity": 0.17212999,\n'
 '  "revenue": 3429388255232,\n'
 '

#### Financial News Tool

In [46]:
### Future integration, need enhancement

In [47]:
class FinancialNewsTool(BaseTool):
    name: str = "financial_news_tool"
    description: str = "Search for recent financial news and market updates"
    # args_schema = FinancialNewsInput
    
    def _run(self, query: str, limit: int = 10) -> str:
        try:
            # Using NewsAPI (you'll need to get a free API key)
            # For demo purposes, we'll use a placeholder implementation
            # In production, integrate with NewsAPI, Alpha Vantage, or similar
            
            # Placeholder implementation using Yahoo Finance news
            news_results = []
            
            # Try to get news for specific symbols if they appear in query
            potential_symbols = [word.upper() for word in query.split() if len(word) <= 5 and word.isalpha()]
            
            for symbol in potential_symbols[:3]:  # Limit to 3 symbols
                try:
                    ticker = yf.Ticker(symbol)
                    news = ticker.news[:limit//len(potential_symbols) if potential_symbols else limit]
                    
                    for article in news:
                        news_results.append({
                            "title": article.get("title", "N/A"),
                            "publisher": article.get("publisher", "N/A"),
                            "link": article.get("link", "N/A"),
                            "published": datetime.fromtimestamp(article.get("providerPublishTime", 0)).strftime("%Y-%m-%d %H:%M:%S") if article.get("providerPublishTime") else "N/A",
                            "summary": article.get("summary", "N/A")[:200] + "..." if len(article.get("summary", "")) > 200 else article.get("summary", "N/A")
                        })
                except:
                    continue
            
            if not news_results:
                return f"No news found for query: {query}"
            
            return json.dumps(news_results[:limit], indent=2)
            
        except Exception as e:
            return f"Error fetching financial news: {str(e)}"

In [48]:
import os
class FinancialNewsTool(BaseTool):
    name: str = "financial_news_tool"
    description: str = "Search for recent financial news and market updates using Alpha Vantage, Yahoo Finance, and NewsAPI"
    # args_schema = FinancialNewsInput

    alpha_vantage_api_key: str
    newsapi_key: Optional[str] = None  # Optional key for NewsAPI

    def _run(self, query: str, limit: int = 10) -> str:
        try:
            news_results = []

            # Extract potential tickers from query
            potential_symbols = [word.upper() for word in query.split() if len(word) <= 5 and word.isalpha()]

            if not potential_symbols:
                return "No valid ticker symbols found in the query."

            # Try each source sequentially
            for symbol in potential_symbols[:3]:
                ticker = yf.Ticker(symbol)
                news_results.extend(self._fetch_alpha_vantage_news(ticker, limit // len(potential_symbols)))

                if len(news_results) < limit:
                    news_results.extend(self._fetch_yahoo_finance_news(ticker, limit // len(potential_symbols)))

                if self.newsapi_key and len(news_results) < limit:
                    news_results.extend(self._fetch_newsapi_news(ticker, limit // len(potential_symbols)))

            if not news_results:
                return f"No news found for query: {query}"

            return json.dumps(news_results[:limit], indent=2)

        except Exception as e:
            return f"Error fetching financial news: {str(e)}"

    def _fetch_alpha_vantage_news(self, symbol: str, limit: int) -> List[Dict]:
        try:
            url = "https://www.alphavantage.co/query"
            params = {
                "function": "NEWS_SENTIMENT",
                "tickers": symbol,
                "apikey": self.alpha_vantage_api_key
            }

            response = requests.get(url, params=params)
            if response.status_code != 200:
                return []

            data = response.json()
            if "feed" not in data or not data["feed"]:
                return []

            articles = []
            for article in data["feed"][:limit]:
                articles.append({
                    "source": "Alpha Vantage",
                    "title": article.get("title", "N/A"),
                    "publisher": article.get("source", "N/A"),
                    "link": article.get("url", "N/A"),
                    "published": article.get("time_published", "N/A"),
                    "summary": article.get("summary", "N/A")[:200] + "..." if len(article.get("summary", "")) > 200 else article.get("summary", "N/A")
                })
            return articles

        except Exception as e:
            print(f"Alpha Vantage error for {symbol}: {e}")
            return []

    def _fetch_yahoo_finance_news(self, symbol: str, limit: int) -> List[Dict]:
        try:
            ticker = yf.Ticker(symbol)
            news = ticker.news[:limit]

            articles = []
            for article in news:
                articles.append({
                    "source": "Yahoo Finance",
                    "title": article.get("title", "N/A"),
                    "publisher": article.get("publisher", "N/A"),
                    "link": article.get("link", "N/A"),
                    "published": datetime.fromtimestamp(article.get("providerPublishTime", 0)).strftime("%Y-%m-%d %H:%M:%S") if article.get("providerPublishTime") else "N/A",
                    "summary": article.get("summary", "N/A")[:200] + "..." if len(article.get("summary", "")) > 200 else article.get("summary", "N/A")
                })
            return articles

        except Exception as e:
            print(f"Yahoo Finance error for {symbol}: {e}")
            return []

    def _fetch_newsapi_news(self, symbol: str, limit: int) -> List[Dict]:
        try:
            url = "https://newsapi.org/v2/everything"
            params = {
                "q": symbol,
                "apiKey": self.newsapi_key,
                "pageSize": limit,
                "sortBy": "publishedAt",
                "language": "en"
            }

            response = requests.get(url, params=params)
            if response.status_code != 200:
                return []

            data = response.json()
            if "articles" not in data or not data["articles"]:
                return []

            articles = []
            for article in data["articles"][:limit]:
                articles.append({
                    "source": "NewsAPI",
                    "title": article.get("title", "N/A"),
                    "publisher": article.get("source", {}).get("name", "N/A"),
                    "link": article.get("url", "N/A"),
                    "published": article.get("publishedAt", "N/A"),
                    "summary": article.get("description", "N/A")[:200] + "..." if len(article.get("description", "")) > 200 else article.get("description", "N/A")
                })
            return articles

        except Exception as e:
            print(f"NewsAPI error for {symbol}: {e}")
            return []

In [49]:
os.getenv("ALPHAVANTAGE_API_KEY")

'0DZP1VNSGT50VW4I'

In [50]:
news_tool = FinancialNewsTool(
    alpha_vantage_api_key=os.getenv("ALPHAVANTAGE_API_KEY"),
    newsapi_key=os.getenv("NEWS_API_KEY")  # Optional if you don't want to use NewsAPI
)
result = news_tool._run("SBIN", limit=5)
print(result)


Yahoo Finance error for yfinance.Ticker object <SBIN>: 'Ticker' object has no attribute 'upper'
No news found for query: SBIN


#### Financial Web Search Tool

In [51]:
import os
os.getenv("SERP_API_KEY")

'cdf009c33c4cb3f472c7cef9b98212c90abb9f76f8ede5ec5c8c5ebb2c79395e'

In [52]:
class FinancialWebSearchTool(BaseTool):
    name: str = "financial_web_search"
    description: str = "Fetch financial data from Google Finance using SerpAPI"
    
    serpapi_api_key: str

    def _run(self, query: str) -> str:
        try:
            url = "https://serpapi.com/search.json"
            params = {"engine": "google_finance", "q": query, "api_key": self.serpapi_api_key}

            resp = requests.get(url, params=params)
            resp.raise_for_status()
            data = resp.json()

            output = {"query": query}

            # Summary section
            if "summary" in data:
                output["summary"] = data["summary"]
            else:
                output["summary"] = None

            # Markets
            output["markets"] = data.get("markets", {})

            # News (if present)
            output["news_results"] = data.get("news_results", [])

            # Financial statements (if present)
            output["financials"] = data.get("financials", [])

            return json.dumps(output, indent=2)

        except Exception as e:
            return f"Error fetching financial data: {str(e)}"

In [53]:
fin_web_search = FinancialWebSearchTool(serpapi_api_key="cdf009c33c4cb3f472c7cef9b98212c90abb9f76f8ede5ec5c8c5ebb2c79395e")
pprint(fin_web_search.invoke({"query":"TATA MOTORS"}))


('{\n'
 '  "query": "TATA MOTORS",\n'
 '  "summary": null,\n'
 '  "markets": {\n'
 '    "us": [\n'
 '      {\n'
 '        "stock": ".DJI:INDEXDJX",\n'
 '        "link": "https://www.google.com/finance/quote/.DJI:INDEXDJX",\n'
 '        "serpapi_link": '
 '"https://serpapi.com/search.json?engine=google_finance&hl=en&q=.DJI%3AINDEXDJX",\n'
 '        "name": "Dow Jones",\n'
 '        "price": 44828.53,\n'
 '        "price_movement": {\n'
 '          "percentage": 0.7735503,\n'
 '          "value": 344.10938,\n'
 '          "movement": "Up"\n'
 '        }\n'
 '      },\n'
 '      {\n'
 '        "stock": ".INX:INDEXSP",\n'
 '        "link": "https://www.google.com/finance/quote/.INX:INDEXSP",\n'
 '        "serpapi_link": '
 '"https://serpapi.com/search.json?engine=google_finance&hl=en&q=.INX%3AINDEXSP",\n'
 '        "name": "S&P 500",\n'
 '        "price": 6279.35,\n'
 '        "price_movement": {\n'
 '          "percentage": 0.8338955,\n'
 '          "value": 51.930176,\n'
 '          "mov